In [124]:
import numpy as np
import datetime
from datetime import date, timedelta
import datetime
import pandas as pd
import math
import itertools
import time

In [ ]:
key = 'crazy'
message = 'secretinformation'

x = sorted(key)

list_key = [0] * len(key)
for i in range(len(key)):
    aux = key.find(x[i]) 
    list_key[aux] = i + 1    
    
#DIVIDIR A MENSAGE, DE ACORDO COM LEN DA KEY
chunks = len(message)
chunk_size = len(key)
sub_strings = [message[i:i+chunk_size] for i in range(0, chunks, chunk_size)]


novo = []
for sub_string in sub_strings:
    aux = ''
    if len(sub_string) < len(list_key):
        sub_string = sub_string + ' '*(len(list_key) - len(sub_string))
    for j in range(1, len(key)+1):
        index = list_key.index(j)
        aux += sub_string[index]
    novo.append(aux)
    
result = ''.join(novo)

In [107]:
key = 'crazy'
x = sorted(key)

['a', 'c', 'r', 'y', 'z']

In [109]:

list_key = [0] * len(key)
for i in range(len(key)):
    aux = key.find(x[i]) 
    list_key[aux] = i + 1    
    

In [110]:
list_key

[2, 3, 1, 5, 4]

In [111]:
message = 'secretinformation'

In [112]:
#DIVIDIR A MENSAGE, DE ACORDO COM LEN DA KEY
chunks = len(message)
chunk_size = len(key)
sub_strings = [message[i:i+chunk_size] for i in range(0, chunks, chunk_size)]

In [93]:
aux = ''
novo = []
for sub_string in sub_strings:
    aux = ''
    for j in range(1, len(key)+1):
        index = list_key.index(j)
        aux += sub_string[index]
    novo.append(aux)

In [94]:
novo

['cseer', 'ntiof', 'armit']

In [116]:

novo = []
for sub_string in sub_strings:
    aux = ''
    if len(sub_string) < len(list_key):
        sub_string = sub_string + ' '*(len(list_key) - len(sub_string))
    for j in range(1, len(key)+1):
        index = list_key.index(j)
        aux += sub_string[index]
    novo.append(aux)
    
result = ''.join(novo)

In [115]:
novo

['cseer', 'ntiof', 'armit', ' on  ']

In [117]:
result

'cseerntiofarmit on  '

In [125]:
day_txt = (date.today() - timedelta(days=2)).strftime('%Y%m%d')

In [126]:
day_txt

'20220910'

In [120]:
import os

In [121]:
day_txt = '2022-08-15'

In [127]:
nc_file = os.path.join('JUNO/data/MUR_daily_fronts_netcdf/' + day_txt + '.nc')

In [128]:
nc_file

'JUNO/data/MUR_daily_fronts_netcdf/20220910.nc'